In [ ]:
import os

# Define the language as a single variable
language = 'jpn'

# Use the language variable to construct the folder path
folder_path = f'new_translationss/{language}'

# List all files in the directory
for root, dirs, files in os.walk(folder_path):
    for file in files:
        print(os.path.join(root, file))


In [ ]:
import os

# Use the language variable to construct the folder path
folder_path = f'new_translationss/{language}'

# Initialize a dictionary to store the line counts for each file
file_line_counts = {}

# Loop through all files in the specified directory
for root, dirs, files in os.walk(folder_path):
    for file in files:
        # Skip .DS_Store and any other hidden files
        if file == '.DS_Store' or file.startswith('.'):
            continue

        file_path = os.path.join(root, file)
        
        # Count lines in the file, ignoring decode errors
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                line_count = sum(1 for line in f)
            # Store the line count in the dictionary
            file_line_counts[file] = line_count
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")

# Check if all files have the same line count
line_counts = list(file_line_counts.values())
all_equal = all(count == line_counts[0] for count in line_counts)

# Print results
if all_equal:
    print("All files have the same number of lines:", line_counts[0])
else:
    print("Mismatch in line counts detected:")
    for file, count in file_line_counts.items():
        print(f"{file}: {count} lines")


In [ ]:
####FOR JAPANESE:
import sacrebleu
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from fugashi import Tagger

# Initialize Fugashi tokenizer
tokenizer = Tagger()

# Create plots folder
plots_folder = os.path.join(folder_path, "plots")
os.makedirs(plots_folder, exist_ok=True)

# Tokenization function using Fugashi
def tokenize_japanese(text):
    return " ".join(token.surface for token in tokenizer(text))

# Function to compute BLEU score
def compute_bleu(reference_file, hypothesis_file):
    with open(reference_file, 'r', encoding='utf-8') as ref_file:
        references = [tokenize_japanese(line) for line in ref_file.read().strip().split('\n')]
    with open(hypothesis_file, 'r', encoding='utf-8') as hyp_file:
        hypotheses = [tokenize_japanese(line) for line in hyp_file.read().strip().split('\n')]
    bleu = sacrebleu.corpus_bleu(hypotheses, [references])
    return bleu.score

# Function to compute chrF score
def compute_chrf(reference_file, hypothesis_file):
    with open(reference_file, 'r', encoding='utf-8') as ref_file:
        references = [tokenize_japanese(line) for line in ref_file.read().strip().split('\n')]
    with open(hypothesis_file, 'r', encoding='utf-8') as hyp_file:
        hypotheses = [tokenize_japanese(line) for line in hyp_file.read().strip().split('\n')]
    chrf = sacrebleu.corpus_chrf(hypotheses, [references])
    return chrf.score

# Function to compute TER score
def compute_ter(reference_file, hypothesis_file):
    with open(reference_file, 'r', encoding='utf-8') as ref_file:
        references = [tokenize_japanese(line) for line in ref_file.read().strip().split('\n')]
    with open(hypothesis_file, 'r', encoding='utf-8') as hyp_file:
        hypotheses = [tokenize_japanese(line) for line in hyp_file.read().strip().split('\n')]
    ter = sacrebleu.corpus_ter(hypotheses, [references])
    return ter.score

# Function to evaluate all metrics and summarize results
def evaluate_all_metrics(reference_file, folder_path, thresholds={'BLEU': 20, 'chrF': 50, 'TER': 60}):
    models = []
    bleu_scores = []
    chrf_scores = []
    ter_scores = []
    bleu_pass_counts = []
    chrf_pass_counts = []
    ter_pass_counts = []
    
    excluded_files = {'eng.txt', f'{language}.txt'}
    hypothesis_files = [f for f in os.listdir(folder_path) if f.endswith('.txt') and f not in excluded_files]
    
    for hypothesis_file in hypothesis_files:
        hyp_path = os.path.join(folder_path, hypothesis_file)
        models.append(hypothesis_file)
        
        bleu_score = compute_bleu(reference_file, hyp_path)
        chrf_score = compute_chrf(reference_file, hyp_path)
        ter_score = compute_ter(reference_file, hyp_path)
        
        bleu_scores.append(bleu_score)
        chrf_scores.append(chrf_score)
        ter_scores.append(ter_score)

        # Count scores exceeding thresholds
        bleu_pass_counts.append(bleu_score >= thresholds['BLEU'])
        chrf_pass_counts.append(chrf_score >= thresholds['chrF'])
        ter_pass_counts.append(ter_score <= thresholds['TER'])  # For TER, lower is better

        print(f"{hypothesis_file}:")
        print(f"  BLEU score: {bleu_score:.2f}")
        print(f"  chrF score: {chrf_score:.2f}")
        print(f"  TER score: {ter_score:.2f}")
    
    # Create DataFrame for detailed analysis
    results_df = pd.DataFrame({
        'Model': models,
        'BLEU Score': bleu_scores,
        'chrF Score': chrf_scores,
        'TER Score': ter_scores,
        'BLEU Above Threshold': bleu_pass_counts,
        'chrF Above Threshold': chrf_pass_counts,
        'TER Below Threshold': ter_pass_counts
    })

    # Summary of thresholds passing rates
    total_files = len(models)
    summary_df = pd.DataFrame({
        'Metric': ['BLEU', 'chrF', 'TER'],
        'Pass Count': [
            sum(bleu_pass_counts),
            sum(chrf_pass_counts),
            sum(ter_pass_counts)
        ],
        'Percentage': [
            sum(bleu_pass_counts) / total_files * 100,
            sum(chrf_pass_counts) / total_files * 100,
            sum(ter_pass_counts) / total_files * 100
        ]
    })
    
    # Display results
    print("\nDetailed Model Performance:")
    print(results_df)
    print("\nSummary of Threshold Passing Rates:")
    print(summary_df)
    
    visualize_metrics(models, bleu_scores, chrf_scores, ter_scores)

# Visualization of metrics
def visualize_metrics(models, bleu_scores, chrf_scores, ter_scores):
    x = np.arange(len(models))
    width = 0.3  # Bar width
    
    fig, ax = plt.subplots(figsize=(12, 8))
    rects1 = ax.bar(x - width, bleu_scores, width, label='BLEU Score')
    rects2 = ax.bar(x, chrf_scores, width, label='chrF Score')
    rects3 = ax.bar(x + width, ter_scores, width, label='TER Score')

    ax.set_xlabel('Models')
    ax.set_title('Translation Performance Metrics')
    ax.set_xticks(x)
    ax.set_xticklabels(models, rotation=45, ha='right')
    ax.legend()

    plt.tight_layout()
    plot_path = os.path.join(plots_folder, f"all_scores_{language}.png")
    plt.savefig(plot_path)
    plt.show()

# Paths
reference_path = f'new_translationss/{language}/{language}.txt'
folder_path = f'new_translationss/{language}'

# Run the evaluation
evaluate_all_metrics(reference_path, folder_path)
